# Linear Algebra Algorithms

## Implementation

In [1]:
import numpy as np
import math

### Matrix general utils

#### Determinant

In [2]:
def determinant(matrix):
    # TODO: pivotting missing here
    
    order = matrix.shape[0]
    upper, lower, _ = lu_decompose(matrix)
    lower_det = 1
    upper_det = 1
    
    for index in range(order):
        lower_det = lower[index, index] * lower_det
        upper_det = upper[index, index] * upper_det
        
    return lower_det * upper_det

#### Simetric check

In [3]:
def check_simetric(matrix):
    order = matrix.shape[0]
    
    for i in range(order):
        for j in range(order):
            if matrix[i,j] != matrix[j,i]:
                return False
 
    return True

#### Simetric and positive defined check

In [4]:
def check_simetric_positive_defined(matrix):
    order = matrix.shape[0]
    
    for i in range(order):
        if matrix[i,i] < 0:
            return False
        
        for j in range(order):
            if matrix[i,j] != matrix[j,i]:
                return False
    
    return True

#### Diagonal dominance check

In [5]:
def check_diagonal_dominance(A):
    order = A.shape[0]
    for diagonal_index in range(order):
        row_sum = 0
        column_sum = 0
        diagonal_item = A[diagonal_index, diagonal_index]
        
        for item_index in range(order):
            row_sum += A[diagonal_index, item_index]
            column_sum += A[item_index, diagonal_index]
        
        if (diagonal_item < row_sum or diagonal_item < column_sum):
            return False
        
        return True

### Decomposition

#### LU Decomposition

In [6]:
# TODO: finish pivoting
# def pivot(matrix):
#     order = matrix.shape[0]
#     for line in range(order-1):
#         if (matrix[line][line]) = 0

def get_matrix_decomposition_step(matrix, step_index):        
        order = matrix.shape[0]
        pivot = matrix[step_index, step_index]
        
        if (pivot == 0):
            raise ValueError('Pivot is 0!', matrix, step_index)
            return
            # TODO: pivotting missing here
#           return get_matrix_decomposition_step(pivot(matrix), step_index)
        
        lower = np.identity(order)
        upper = np.identity(order)
        
        for line_index in range (step_index + 1, order):
            element = matrix[line_index, step_index] / pivot
            
            lower[line_index, step_index] = element
            upper[line_index, step_index] = - element
            
        return lower, upper
    
def lu_decompose(matrix):
    order = matrix.shape[0]
    upper = matrix
    upper_combination = np.identity(order)
    lower = np.identity(order)
    
    for step_index in range (order - 1):
        lower_step, upper_step = get_matrix_decomposition_step(upper, step_index)
        upper_combination = upper_step @ upper_combination
        lower = lower_step@lower
        upper = upper_step@upper

    return lower, upper, upper_combination


#### Cholesky decomposition

In [7]:
def cholesky_decompose(A):
    order = A.shape[0]

    L = np.zeros((order, order))

    for i in range(order):
        for k in range(i+1):
            tmp_sum = sum(L[i,j] * L[k,j] for j in range(k))
            
            if (i == k):
                L[i,k] = math.sqrt(A[i,i] - tmp_sum)
            else:
                L[i,k] = (1.0 / L[k,k] * (A[i,k] - tmp_sum))
    return L

#### Gauss elimination

In [8]:
def gauss_elimination(matrix):
    _, upper, upper_combination = lu_decompose(matrix)
    return upper, upper_combination

### AX = B solving

#### By gauss elimination

In [9]:
def solve_equation_gauss_elimination(A, B):
    order = A.shape[0]
    last_index = order-1

    A, combination_matrix = gauss_elimination(A)
    B = combination_matrix @ B
    
    X = np.zeros(order)
    
    X[last_index] = B[last_index] / A[last_index, last_index]
        
    # TODO: this doesn't need to be backwards ???
    for x_index in range(last_index - 1, -1, -1): # from n-1 to 0 
        backwards_sum = 0
        for sum_index in range(last_index, x_index, -1):
            backwards_sum += A[x_index, sum_index] * X[sum_index] 
        
        X[x_index] = (B[x_index] - backwards_sum) / A[x_index, x_index]
        
    return X

#### By Jacobi method

In [10]:
def solve_equation_jacobi(A, B, tolerance=0.000001):
    order = A.shape[0]
    residue = tolerance + 1
    X = np.ones(order)
    
    if (not check_diagonal_dominance(A)):
        return -1
    
    while(residue > tolerance):
        currentX = np.ones(order)
        
        for x_index in range (order):
            subtrahend = 0
            
            for sum_index in range (order):
                if (sum_index == x_index):
                    continue
                subtrahend += A[x_index, sum_index] * X[sum_index]
            
            currentX[x_index] = (B[x_index] - subtrahend) / A[x_index, x_index]
        
        residue = np.linalg.norm(currentX - X, ord=2) / np.linalg.norm(currentX, ord=2)
        X = currentX
    
    return X
        

#### By Gauss-seidel method

In [11]:
def solve_equation_gauss_seidel(A, B, tolerance=0.000001):
    order = A.shape[0]
    residue = tolerance + 1
    X = np.ones(order)
    
    if (not check_simetric_positive_defined(A) and not check_diagonal_dominance(A)):
        return -1
    
    while(residue > tolerance):
        currentX = np.ones(order)
        
        for i in range (order):
            subtrahend = 0
            
            for j in range (i):
                subtrahend += A[i, j] * currentX[j]
    
            for j in range (i + 1, order):
                subtrahend += A[i, j] * X[j]

            currentX[i] = (B[i] - subtrahend) / A[i, i]
        
        residue = np.linalg.norm(currentX - X, ord=2) / np.linalg.norm(currentX, ord=2)
        X = currentX
    
    return X

### Eigenvalues and eigenvectors

#### Power Method

Returns the greatest eigenvector and its corresponding eigenvalue

In [12]:
def eigen_power_method(A, tolerance=0.000001):
    order = A.shape[0]
    
    eigenvector = np.ones(order)
    residue = tolerance + 1
    prev_eigenvalue = 1
    
    while(tolerance < residue):   
        Y = A @ eigenvector
        
        eigenvalue = Y[0]
        eigenvector = Y / eigenvalue
        
        residue = (eigenvalue - prev_eigenvalue) / eigenvalue
        prev_eigenvalue = eigenvalue

    return eigenvector, eigenvalue 

#### Jacobi method

Returns aproximated eigenvalues and eigenvectors. Matrix must be simetric.

In [24]:
def eigen_jacobi(A, tolerance=0.00000001):
    order = A.shape[0]
    if not check_simetric(A):
        return -1
        
    X = np.identity(order)
    residue = tolerance + 1
    
    while(residue > tolerance):
        # Finds greater absolute value position outside diagonal
        greatest_el = 0
        for i in range(order):
            for j in range(order):
                if i != j and abs(A[i,j]) > abs(greatest_el):
                    greatest_el_pos = (i, j)
                    greatest_el = A[i,j]
        
        # Compute phi for P matrix
        i, j = greatest_el_pos
        if (A[i,i] != A[j,j]):
            phi = 1/2 * np.arctan(2 * A[i,j] / (A[i,i] - A[j,j]))
        else:
            phi = np.pi / 4
        
        # Compute P matrix
        P = np.identity(order)
        P[i,i] = np.cos(phi)
        P[j,j] = np.cos(phi)
        P[j,i] = np.sin(phi)
        P[i,j] = -np.sin(phi)
        
        # Next iteration A matrix
        residue = abs(greatest_el)
        A = np.transpose(P) @ A @ P
        X = X @ P
        
    eigenvalues = np.zeros(order)
    
    # Eigenvalues are the values in the main diagonal
    for i in range(order):
        for j in range(order):
            if i == j:
                eigenvalues[i] = A[i,j]

    eigenvectors = X 
    
    return eigenvectors, eigenvalues